In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

sf_fire_file = "/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"

In [0]:
#how to work with files on Databricks https://docs.databricks.com/files/index.html

#inspect the data 
%fs head databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv

[Truncated to first 65536 bytes]
Call Number,Unit ID,Incident Number,CallType,Call Date,Watch Date,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,OrigPriority,Priority,Final Priority,ALS Unit,Call Type Group,NumAlarms,UnitType,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhood,Location,RowID,Delay
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,"",1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,"",1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666
20110016,E03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,7,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E03,2.683333333333333
20110016,E38,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:17 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,1,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E38,2.1
20110016,E41,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,ENGINE,8,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E41,2.716666666666667
20110016,M03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:38 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,true,"",1,MEDIC,10,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-M03,2.7666666666666666
20110016,RS1,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:57 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,RESCUE SQUAD,9,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-RS1,3.2666666666666666
20110016,T03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,TRUCK,5,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-T03,3.2333333333333334
20110016,T13,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:44 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,"",1,TRUCK,2,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-T13,2.95
20110017,M13,2003236,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:27:14 AM,700 Block of FOLSOM ST,SF,94107,B03,01,2176,1,1,2,true,"",1,MEDIC,1,3,6,South of Market,"(37.7831757914875, -122.399603541822)",020110017-M13,6.933333333333334
20110019,M36,2003238,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:28:30 AM,9TH ST/LARKIN ST,SF,94103,B02,36,2337,1,1,2,true,"",1,MEDIC,1,2,6,South of Market,"(37.7774938436091, -122.41629170277)",020110019-M36,6.116666666666666
20110020,E38,2003239,Outside Fire,01/11/2002,01/10/2002,Other,01/11/2002 02:15:33 AM,3000 Block of WEBSTER ST,SF,94123,B04,16,3462,3,

In [0]:
#define schema
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),      
                     StructField('WatchDate', StringType(), True),
                     StructField('CallFinalDisposition', StringType(), True),
                     StructField('AvailableDtTm', StringType(), True),
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('Zipcode', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('Neighborhood', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True),
                     StructField('Delay', FloatType(), True)])

In [0]:
fire_sdf = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)

In [0]:
#Best practice for cache(), count(), and take()   https://learn.microsoft.com/en-us/azure/databricks/kb/scala/best-practice-cache-count-take

#Cache the DataFrame
fire_sdf.cache()

Out[6]: DataFrame[CallNumber: int, UnitID: string, IncidentNumber: int, CallType: string, CallDate: string, WatchDate: string, CallFinalDisposition: string, AvailableDtTm: string, Address: string, City: string, Zipcode: int, Battalion: string, StationArea: string, Box: string, OriginalPriority: string, Priority: string, FinalPriority: int, ALSUnit: boolean, CallTypeGroup: string, NumAlarms: int, UnitType: string, UnitSequenceInCallDispatch: int, FirePreventionDistrict: string, SupervisorDistrict: string, Neighborhood: string, Location: string, RowID: string, Delay: float]

In [0]:
fire_sdf.count()

Out[7]: 4380660

In [0]:
fire_sdf.printSchema()

root
-- CallNumber: integer (nullable = true)
-- UnitID: string (nullable = true)
-- IncidentNumber: integer (nullable = true)
-- CallType: string (nullable = true)
-- CallDate: string (nullable = true)
-- WatchDate: string (nullable = true)
-- CallFinalDisposition: string (nullable = true)
-- AvailableDtTm: string (nullable = true)
-- Address: string (nullable = true)
-- City: string (nullable = true)
-- Zipcode: integer (nullable = true)
-- Battalion: string (nullable = true)
-- StationArea: string (nullable = true)
-- Box: string (nullable = true)
-- OriginalPriority: string (nullable = true)
-- Priority: string (nullable = true)
-- FinalPriority: integer (nullable = true)
-- ALSUnit: boolean (nullable = true)
-- CallTypeGroup: string (nullable = true)
-- NumAlarms: integer (nullable = true)
-- UnitType: string (nullable = true)
-- UnitSequenceInCallDispatch: integer (nullable = true)
-- FirePreventionDistrict: string (nullable = true)
-- SupervisorDistrict: string (nullable = true)
-- Neighborhood: string (nullable = true)
-- Location: string (nullable = true)
-- RowID: string (nullable = true)
-- Delay: float (nullable = true)

In [0]:
#show 50 rows with a quick visual - average delay by visual
display(fire_sdf.limit(50))

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.3166666
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166667
20110016,E03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,ENGINE,7,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E03,2.6833334
20110016,E38,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:17 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,ENGINE,1,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E38,2.1
20110016,E41,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,ENGINE,8,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-E41,2.7166667
20110016,M03,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:38 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,true,null,1,MEDIC,10,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-M03,2.7666667
20110016,RS1,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:46:57 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,RESCUE SQUAD,9,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-RS1,3.2666667


Output can only be rendered in Databricks

In [0]:
#filter out "Medical Incident" call types

filtered_fire_sdf = (
    fire_sdf.select("IncidentNumber","AvailableDtTm","CallType")
            .where(col("CallType") != "Medical Incident")
)

filtered_fire_sdf.show(5, truncate=False)

+--------------+----------------------+--------------+
IncidentNumber|AvailableDtTm |CallType |
+--------------+----------------------+--------------+
2003235 |01/11/2002 01:47:00 AM|Structure Fire|
2003235 |01/11/2002 01:51:54 AM|Structure Fire|
2003235 |01/11/2002 01:47:00 AM|Structure Fire|
2003235 |01/11/2002 01:47:00 AM|Structure Fire|
2003235 |01/11/2002 01:51:17 AM|Structure Fire|
+--------------+----------------------+--------------+
only showing top 5 rows

In [0]:
fire_sdf.select("CallType").where(col("CallType").isNotNull()).distinct().count()

Out[11]: 32

In [0]:
fire_sdf.select("CallType").where(col("CallType").isNotNull()).distinct().show(truncate=False)

+-------------------------------+
CallType |
+-------------------------------+
Elevator / Escalator Rescue |
Aircraft Emergency |
Alarms |
Odor (Strange / Unknown) |
Citizen Assist / Service Call |
HazMat |
Oil Spill |
Vehicle Fire |
Suspicious Package |
Other |
Outside Fire |
Assist Police |
Gas Leak (Natural and LP Gases)|
Water Rescue |
Electrical Hazard |
Structure Fire |
Industrial Accidents |
Medical Incident |
Fuel Spill |
Smoke Investigation (Outside) |
+-------------------------------+
only showing top 20 rows

#title

In [0]:
display(fire_sdf.select("Delay").where(col("Delay") > 10))

Delay
77.333336
18.066668
12.733334
21.116667
10.3
11.066667
28.25
11.383333
10.533334
10.566667


Output can only be rendered in Databricks

In [0]:
(fire_sdf
 .select("CallType").where(col("CallType").isNotNull())
 .groupBy("CallType")
 .count()
 .orderBy("count", ascending=False)
 .show(n=10, truncate=False))

+-------------------------------+-------+
CallType |count |
+-------------------------------+-------+
Medical Incident |2843475|
Structure Fire |578998 |
Alarms |483518 |
Traffic Collision |175507 |
Citizen Assist / Service Call |65360 |
Other |56961 |
Outside Fire |51603 |
Vehicle Fire |20939 |
Water Rescue |20037 |
Gas Leak (Natural and LP Gases)|17284 |
+-------------------------------+-------+
only showing top 10 rows

In [0]:
display(fire_sdf
 .select("CallType").where(col("CallType").isNotNull())
 .groupBy("CallType")
 .count()
 .orderBy("count", ascending=False)
 .limit(10))

CallType,count
Medical Incident,2843475
Structure Fire,578998
Alarms,483518
Traffic Collision,175507
Citizen Assist / Service Call,65360
Other,56961
Outside Fire,51603
Vehicle Fire,20939
Water Rescue,20037
Gas Leak (Natural and LP Gases),17284


Output can only be rendered in Databricks

In [0]:
display(fire_sdf
 .select(col("ZipCode").cast('string'))
 .where(col("CallType").isNotNull())
 .groupBy("Zipcode")
 .count()
 .orderBy("count", ascending=False))

Zipcode,count
94102,543425
94103,521881
94110,370467
94109,365325
94124,229764
94112,207329
94115,196951
94107,174787
94122,159162
94133,157593


Output can only be rendered in Databricks

In [0]:
display(fire_sdf
        .select("Zipcode","Neighborhood")
        .where((col("Zipcode") == 94102) | (col("Zipcode") == 94103))
        .orderBy("Zipcode","Neighborhood")
        .distinct())


Zipcode,Neighborhood
94103,South of Market
94102,Financial District/South Beach
94102,South of Market
94103,Castro/Upper Market
94102,Hayes Valley
94103,Financial District/South Beach
94102,Tenderloin
94102,Nob Hill
94102,Mission
94103,Mission


In [0]:
fire_sdf.select(sum("NumAlarms"), avg("Delay"), min("Delay"), max("Delay")).show()


+--------------+-----------------+-----------+----------+
sum(NumAlarms)| avg(Delay)| min(Delay)|max(Delay)|
+--------------+-----------------+-----------+----------+
 4403441|3.902170335891614|0.016666668| 1879.6167|
+--------------+-----------------+-----------+----------+

In [0]:
(fire_sdf
 .filter(year(to_date(col("CallDate"), "MM/dd/yyy")) == 2018)
 .groupBy("Neighborhood")
 .agg(avg("Delay").alias("AverageDelay"))
 .show()
)

+--------------------+------------------+
 Neighborhood| AverageDelay|
+--------------------+------------------+
 Inner Sunset| 4.387634401598253|
 Haight Ashbury| 3.339983471500102|
 Lincoln Park| 5.50080411449859|
 Japantown| 3.79634004429778|
 None| 4.000947864982189|
 North Beach| 3.763534878129052|
 Lone Mountain/USF| 3.973739401408667|
 Western Addition| 3.629706622189915|
 Bernal Heights| 4.383430825528679|
 Mission Bay|3.8257928749673384|
 Hayes Valley| 3.66146959354379|
Financial Distric...| 4.247475881326647|
 Lakeshore| 4.308008610621571|
Bayview Hunters P...| 4.263592307821738|
 Visitacion Valley|3.8855222090189208|
 Inner Richmond|3.8529741869977685|
 Nob Hill|3.6308939546554555|
Oceanview/Merced/...|3.7002477010845922|
 Outer Richmond|3.9601809738750022|
 Treasure Island| 5.514422900772491|
+--------------------+------------------+
only showing top 20 rows